In [2]:
import json
import sqlite3
import pandas as pd
import os

In [3]:
class SpiderDataset:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.train_data = self.load_json('train_spider.json')
        self.dev_data = self.load_json('dev.json')
        self.schema_data = self.load_json('tables.json')

    def load_json(self, filename):
        with open(os.path.join(self.dataset_path, filename), 'r') as file:
            return json.load(file)

    def get_schema_by_db_id(self, db_id):
        for schema in self.schema_data:
            if schema['db_id'] == db_id:
                return schema
        return None

    def get_db_path(self, db_id):
        return os.path.join(self.dataset_path, 'database', db_id, f'{db_id}.sqlite')

    def execute_query(self, db_id, query):
        db_path = self.get_db_path(db_id)
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        try:
            cursor.execute(query)
            result = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(result, columns=columns)
        except sqlite3.Error as e:
            print(f"Execution error: {e}")
            return None
        finally:
            cursor.close()
            conn.close()

In [4]:
dataset = SpiderDataset('./dataset')

# Load a sample query from dev dataset
sample_query = dataset.train_data[1]
question = sample_query['question']
gold_sql = sample_query['query']
db_id = sample_query['db_id']
schema = dataset.get_schema_by_db_id(db_id)

print("Question:", question)
print("Gold SQL:", gold_sql)
print("Schema:", schema)

# Executing a query on the Spider sample DB
result_df = dataset.execute_query(db_id, gold_sql)
if result_df is not None:
    print(result_df.head())
else:
    print("Query execution failed.")

df_final = pd.DataFrame(columns=['gold', 'db_id'])
for i in range(100):
    sample_query = dataset.train_data[i]
    df_final.loc[i] = [sample_query['query'], sample_query['db_id']]

    
df_final

Question: List the name, born state and age of the heads of departments ordered by age.
Gold SQL: SELECT name ,  born_state ,  age FROM head ORDER BY age
Schema: {'column_names': [[-1, '*'], [0, 'department id'], [0, 'name'], [0, 'creation'], [0, 'ranking'], [0, 'budget in billions'], [0, 'num employees'], [1, 'head id'], [1, 'name'], [1, 'born state'], [1, 'age'], [2, 'department id'], [2, 'head id'], [2, 'temporary acting']], 'column_names_original': [[-1, '*'], [0, 'Department_ID'], [0, 'Name'], [0, 'Creation'], [0, 'Ranking'], [0, 'Budget_in_Billions'], [0, 'Num_Employees'], [1, 'head_ID'], [1, 'name'], [1, 'born_state'], [1, 'age'], [2, 'department_ID'], [2, 'head_ID'], [2, 'temporary_acting']], 'column_types': ['text', 'number', 'text', 'text', 'number', 'number', 'number', 'number', 'text', 'text', 'number', 'number', 'number', 'text'], 'db_id': 'department_management', 'foreign_keys': [[12, 7], [11, 1]], 'primary_keys': [1, 7, 11], 'table_names': ['department', 'head', 'managem

,gold,db_id
0,SELECT count(*) FROM head WHERE age > 56,department_management
1,"SELECT name , born_state , age FROM head ORD...",department_management
2,"SELECT creation , name , budget_in_billions ...",department_management
3,"SELECT max(budget_in_billions) , min(budget_i...",department_management
4,SELECT avg(num_employees) FROM department WHER...,department_management
...,...,...
95,SELECT course_name FROM courses ORDER BY cours...,student_assessment
96,SELECT course_name FROM courses ORDER BY cours...,student_assessment
97,SELECT first_name FROM people ORDER BY first_name,student_assessment
98,SELECT first_name FROM people ORDER BY first_name,student_assessment


In [12]:
df_final.to_csv("gold.tsv", sep='\t', index=False, header=False)

In [44]:
df5 = pd.read_csv("gold.csv")
df5

ParserError: Error tokenizing data. C error: Expected 3 fields in line 3, saw 5


In [5]:
df2 = pd.read_csv("text2sql_results_fewshot.csv")
#df1 = pd.read_csv("control_results.csv")
#df2 = df2.iloc[2:]
#df2 = df2.reset_index(drop=True)
df2

,query_id,question,schema,agent_1_output,agent_2_output,agent_3a_output_1,agent_3b_output_1,execution_output_1,agent_3c_output_1,agent_3a_output_2,agent_3b_output_2,execution_output_2,agent_3c_output_2,agent_3a_output_3,agent_3b_output_3,execution_output_3,agent_3c_output_3,final_query
0,0,How many heads of the departments are older th...,"{'column_names': [[-1, '*'], [0, 'department i...",Tables - head; Columns - age; Ambiguities - None,SELECT COUNT(*) FROM head WHERE age > 56;,The SQL query doesn't have any syntax issues.,{},"(True, COUNT(*)\n0 5)",SATISFACTORY SQL QUERY GENERATED\nSELECT COUNT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SELECT COUNT(*) FROM head WHERE age > 56;
1,1,"List the name, born state and age of the heads...","{'column_names': [[-1, '*'], [0, 'department i...","Tables - head, management; Columns - name, bor...","SELECT head.name, head.born_state, head.age FR...","The SQL query syntax is correct, there are no ...",{},"(True, name born_state ag...",SATISFACTORY SQL QUERY GENERATED\nSELECT head....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SELECT head.name, head.born_state, head.age FR..."
2,2,"List the creation year, name and budget of eac...","{'column_names': [[-1, '*'], [0, 'department i...","Tables - department; Columns - creation, name,...","SELECT creation, name, budget in billions FROM...","Issue: The phrase ""in billions"" is not a valid...",{},"(False, 'Query execution failed.')",UNSATISFACTORY SQL QUERY GENERATED\n\nCorrecte...,"The given SQL query: SELECT creation, name, `b...",{},"(False, 'Query execution failed.')",SATISFACTORY SQL QUERY GENERATED\nSELECT creat...,NaN,NaN,NaN,NaN,"Corrected SQL Query: SELECT creation, name, `b..."
3,3,What are the maximum and minimum budget of the...,"{'column_names': [[-1, '*'], [0, 'department i...",Tables - department; Columns - budget in billi...,"SELECT MAX(`budget in billions`), MIN(`budget ...","The SQL query syntax appears to be correct, th...",{},"(False, 'Query execution failed.')",SATISFACTORY SQL QUERY GENERATED\nSELECT MAX(`...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SELECT MAX(`budget in billions`), MIN(`budget ..."
4,4,What is the average number of employees of the...,"{'column_names': [[-1, '*'], [0, 'department i...","Tables - department; Columns - num employees, ...",SELECT AVG(`num employees`) FROM department WH...,There are no SQL syntax issues in the given qu...,{},"(False, 'Query execution failed.')",SATISFACTORY SQL QUERY GENERATED\n\nSELECT AVG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SELECT AVG(`num employees`) FROM department WH...
5,5,What are the names of the heads who are born o...,"{'column_names': [[-1, '*'], [0, 'department i...","Tables - head; Columns - name, born state; Amb...",SELECT name FROM head WHERE born_state != 'Cal...,The SQL query syntax is correct.,{},"(True, name\n0 Tiger ...",SATISFACTORY SQL QUERY GENERATED\nSELECT name ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SELECT name FROM head WHERE born_state != 'Cal...
6,6,What are the distinct creation years of the de...,"{'column_names': [[-1, '*'], [0, 'department i...","Tables - department, head, management; Columns...",SELECT DISTINCT department.creation \nFROM dep...,The issue is with the space in the column name...,"{""department.department id"": ""department.depar...","(False, 'Query execution failed.')",UNSATISFACTORY SQL QUERY GENERATED\n\nSELECT D...,The SQL query does not have any syntax issues.,{},"(True, Creation\n0 1903)",SATISFACTORY SQL QUERY GENERATED\nSELECT DISTI...,NaN,NaN,NaN,NaN,SELECT DISTINCT department.creation \nFROM dep...
7,7,What are the names of the states where at leas...,"{'column_names': [[-1, '*'], [0, 'department i...",Tables - head; Columns - born_state; Ambiguiti...,SELECT born_state FROM head GROUP BY born_stat...,The SQL query syntax appears to be correct wit...,{},"(True, born_state\n0 California)",SATISFACTORY SQL QUERY GENERATED\nSELECT born_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SELECT born_state FROM head GROUP BY born_stat...
8,8,In which year were most departm

In [14]:
df_control = pd.DataFrame({
    'prediction': df1['query'],
    'db_id': df_final['db_id']
})

In [15]:
df_control['prediction'] = df_control['prediction'].str.replace('\n', ' ', regex=False)

In [16]:
df_control

,prediction,db_id
0,SELECT COUNT(*) FROM head WHERE age > 56,department_management
1,"SELECT head.name, head.born_state, head.age FR...",department_management
2,"SELECT creation, name, budget in billions FROM...",department_management
3,"SELECT MAX(`budget in billions`), MIN(`budget ...",department_management
4,SELECT AVG(`num employees`) FROM department WH...,department_management
...,...,...
95,SELECT course_name FROM courses ORDER BY cours...,student_assessment
96,SELECT course_name FROM courses ORDER BY cours...,student_assessment
97,SELECT `first name` FROM people ORDER BY `firs...,student_assessment
98,SELECT `first name` FROM people ORDER BY `firs...,student_assessment


In [7]:
df_text2sql = pd.DataFrame({
    'prediction': df2['final_query'],
    'db_id': df_final['db_id']
})
#df_text2sql['prediction'] = df_text2sql['prediction'].str.replace('\n', ' ', regex=False)

In [11]:
df_text2sql = df_text2sql.iloc[0:30]
df_text2sql

,prediction,db_id
0,SELECT COUNT(*) FROM head WHERE age > 56;,department_management
1,"SELECT head.name, head.born_state, head.age FR...",department_management
2,"Corrected SQL Query: SELECT creation, name, `b...",department_management
3,"SELECT MAX(`budget in billions`), MIN(`budget ...",department_management
4,SELECT AVG(`num employees`) FROM department WH...,department_management
5,SELECT name FROM head WHERE born_state != 'Cal...,department_management
6,SELECT DISTINCT department.creation \nFROM dep...,department_management
7,SELECT born_state FROM head GROUP BY born_stat...,department_management
8,"SELECT creation, COUNT(*) FROM department GROU...",department_management
9,"SELECT department.name, department.num_employe...",department_management


In [12]:
df_text2sql.to_csv("text2sql_fewshot.tsv", sep='\t', index=False, header=False)

In [40]:
df_control.to_csv("control.tsv", sep='\t', index=False, header=False)

In [41]:
print(df_control.iloc[28])

prediction    SELECT COUNT(DISTINCT status) FROM city
db_id                                            farm
Name: 28, dtype: object
